<a href="https://colab.research.google.com/github/Mikophi/Catora/blob/main/Capstone_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Capstone model

In [ ]:
import urllib.request
import zipfile
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
urllib.request.urlretrieve(
    inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model =  InceptionV3(
    input_shape = (150,150,3),
    include_top = False,
    weights = local_weights_file
)


for layer in pre_trained_model.layers:
        layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7').output

In [ ]:
data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
local_file = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/horse-or-human')

data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
local_file = 'validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/validation-horse-or-human')
zip_ref.close()

In [ ]:
data_train_dir = 'data/horse-or-human'
data_val_dir = 'data/validation-horse-or-human'
training_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode='nearest')

train_generator=training_datagen.flow_from_directory(
        data_train_dir,
        batch_size=32,
        class_mode='binary',
        target_size=(150,150)
    )

validation_datagen = ImageDataGenerator( rescale = 1.0/255 )

validation_generator = validation_datagen.flow_from_directory(
        data_val_dir,
        batch_size=32,
        class_mode='binary',
        target_size=(150,150)
    )

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
x = layers.Flatten()(last_layer)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.0001),
    loss='binary_crossentropy',
    metrics=['acc'])

model.fit(
    train_generator,
    validation_data = validation_generator,
    epochs = 5,
    verbose = 2
    )

Epoch 1/5
33/33 - 29s - loss: 0.1563 - acc: 0.9338 - val_loss: 0.0164 - val_acc: 1.0000 - 29s/epoch - 888ms/step
Epoch 2/5
33/33 - 12s - loss: 0.0302 - acc: 0.9922 - val_loss: 0.0804 - val_acc: 0.9688 - 12s/epoch - 372ms/step
Epoch 3/5
33/33 - 12s - loss: 0.0148 - acc: 0.9971 - val_loss: 0.0276 - val_acc: 0.9844 - 12s/epoch - 368ms/step
Epoch 4/5
33/33 - 12s - loss: 0.0192 - acc: 0.9912 - val_loss: 0.0054 - val_acc: 1.0000 - 12s/epoch - 367ms/step
Epoch 5/5
33/33 - 13s - loss: 0.0153 - acc: 0.9951 - val_loss: 0.0026 - val_acc: 1.0000 - 13s/epoch - 383ms/step


In [ ]:
import numpy as np

from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

user_input=files.upload()

for fn in user_input.keys():

  # predicting images
  path='/content/' + fn
  gmbr=load_img(path, target_size=(150, 150))

  x=img_to_array(gmbr)
  x /= 255
  x=np.expand_dims(x, axis=0)
  gambar_pred = np.vstack([x])

  classes = model.predict(gambar_pred, batch_size=10)

  print(classes[0])

  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")


TypeError: ignored

In [ ]:
model.save("Capstone_model.h5")